# Baseline dataset

In [3]:
import pandas as pd

In [4]:
from pymongo import MongoClient

def load_data_from_mongo(collection_name, fields):

    MONGO_URI = "mongodb://localhost:27017/"
    
    client = MongoClient(MONGO_URI) # Hosted with Docker

    db = client["rightmove"]

    # Access collection
    collection = db[collection_name]
    
    # fields = {"propertyImages": 1, "id": 1}
    query = {}
    
    # Fetch data from the collection
    data = collection.find(query, fields)
    
    # Convert to Pandas DataFrame
    df = pd.DataFrame(list(data))

    return df

In [ ]:
fields = {"id": 1, "bedrooms": 1, "bathrooms":1, "location":1, "price":1, "feature_list": 1, "commercial": 1, "development": 1, "students": 1, "summary": 1}
df = load_data_from_mongo("properties", fields)

In [104]:
df = df.drop_duplicates(subset=['id'])

### Preprocessing

In [ ]:
def convert_frequencies(x):
    frequency = x['frequency']
    price = x['amount']
    
    if frequency == 'monthly':
        return price * 12
    elif frequency == 'weekly':
        return (price / 7) * 365
    elif frequency == 'daily':
        return price * 365
    elif frequency == 'quarterly':
        return price * 4
    else: # Yearly
        return price


def remove_anamolies(df):
    df = df[df['bedrooms'] < 50]
    df = df[df['bathrooms'] < 18]
    df = df[df['price'] < 1_000_000]

    return df

def merge_text(x):
    summary = x[0]
    feature_list = x[1]

    if feature_list:
        feature_list_joined = ', '.join(feature_list)
    else:
        feature_list_joined = ''

    return feature_list_joined + ' , ' + summary

df['longitude'] = df['location'].apply(lambda x: x['longitude'])
df['latitude'] = df['location'].apply(lambda x: x['latitude'])

df = df.drop(columns=['location'])
df['price'] = df['price'].apply(convert_frequencies)

df['commercial'] = df['commercial'].apply(lambda x: 1 if x else 0)
df['development'] = df['development'].apply(lambda x: 1 if x else 0)
df['students'] = df['students'].apply(lambda x: 1 if x else 0)

df['text'] = df[['summary', 'feature_list']].apply(merge_text, axis=1)

In [113]:
df

,_id,id,bedrooms,bathrooms,summary,price,commercial,development,students,feature_list,longitude,latitude,text
0,656dfa586037a12e52f660b6,142473758,2,1.0,SPRING TO KING'S CRESCENT! Bright and spacio...,8400.0,0,0,1,"[Modern throughout, Bright Spacious Rooms, Exc...",-2.100750,57.156601,"Modern throughout, Bright Spacious Rooms, Exce..."
1,656dfa586037a12e52f660b7,86375754,2,1.0,Belvoir are pleased to offer for rental this f...,6000.0,0,0,0,"[Ultrafast Broadband Available, Green Space wi...",-2.116880,57.166310,"Ultrafast Broadband Available, Green Space wit..."
2,656dfa586037a12e52f660b8,141848477,2,1.0,We offer For Let this LUXURY 2 BEDROOM FULLY F...,10200.0,0,0,0,"[Stylish 2 Bedroom First Floor Apartment, Spac...",-2.085600,57.152140,"Stylish 2 Bedroom First Floor Apartment, Spaci..."
3,656dfa586037a12e52f660b9,142066406,2,1.0,Property Reference: 1898640.We are pleased to ...,9000.0,0,0,0,"[No Agent Fees, Students Can Enquire, Property...",-2.098261,57.145840,"No Agent Fees, Students Can Enquire, Property ..."
4,656dfa586037a12e52f660ba,142557041,2,2.0,This ground floor property comprises of; entra...,10200.0,0,0,0,"[Double glazing, Shower, Washing machine, Tele...",-2.095850,57.153930,"Double glazing, Shower, Washing machine, Telev..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
62939,65a2c346a6cfbd1892684015,141514433,3,2.0,A fabulous three bedroom two bathroom apartm...,24000.0,0,0,0,"[Spectacular City Centre Living, Three Double ...",-1.895690,52.487440,"Spectacular City Centre Living, Three Double B..."
62940,65a2c346a6cfbd1892684016,143612339,1,NaN,A furnished room to let in a block on the sou...,5580.0,0,0,0,"[Top Floor, Double Room, South Cliff, Shared G...",-0.404352,54.275110,"Top Floor, Double Room, South Cliff, Shared Ga..."
62941,65a2c346a6cfbd1892684017,138669044,6,0.0,CPH are DELIGHTED to offer to the RENTAL MARKE...,28800.0,0,0,0,"[SIX BEDROOM DETACHED HOUSE, OFF STREET PARKIN...",-0.442809,54.296051,"SIX BEDROOM DETACHED HOUSE, OFF STREET PARKING..."
62942,65a2c346a6cfbd1892684018,143622011,3,2.0,"11, Brindle Way is a lovely three bedroom semi...",9600.0,0,0,0,"[Semi-detached three bedroom family home, Kitc...",-0.773246,54.135269,"Semi-detached three bedroom family home, Kitch..."


In [114]:
df = df.drop(columns=['summary', 'feature_list'])

## Add Walk Score

In [116]:
fields = {"id": 1, "scores":1}
walk_score_df = load_data_from_mongo("walk_scores", fields)

In [117]:
walk_score_df = walk_score_df.drop_duplicates(subset=['id'])

In [118]:
full_df = df.merge(walk_score_df[['id', 'scores']], on='id')

### Preprocess Walk scores

In [130]:
full_df['walk_score'] = full_df['scores'].apply(lambda x: x['walk_score'])
full_df['restaurants'] = full_df['scores'].apply(lambda x: x['restaurants'])
full_df['shopping'] = full_df['scores'].apply(lambda x: x['shopping'])
full_df['coffee'] = full_df['scores'].apply(lambda x: x['coffee'])
full_df['banks'] = full_df['scores'].apply(lambda x: x['banks'])
full_df['parks'] = full_df['scores'].apply(lambda x: x['parks'])
full_df['schools'] = full_df['scores'].apply(lambda x: x['schools'])
full_df['books'] = full_df['scores'].apply(lambda x: x['books'])
full_df['entertainment'] = full_df['scores'].apply(lambda x: x['entertainment'])
full_df['grocery'] = full_df['scores'].apply(lambda x: x['grocery'])

In [133]:
full_df = full_df.drop(columns=['id', '_id', 'scores'])

In [143]:
full_df.head()

,bedrooms,bathrooms,price,commercial,development,students,longitude,latitude,text,walk_score,restaurants,shopping,coffee,banks,parks,schools,books,entertainment,grocery
0,2,1.0,8400.0,0,0,1,-2.100750,57.156601,"Modern throughout, Bright Spacious Rooms, Exce...",70.121480,2.247757,0.729202,1.900837,1.272753e-01,0.613228,0.880319,0.957964,6.415095e-02,2.992232
1,2,1.0,6000.0,0,0,0,-2.116880,57.166310,"Ultrafast Broadband Available, Green Space wit...",34.980814,0.248922,0.034009,0.169559,4.073974e-03,0.117854,0.997014,0.681047,3.699971e-39,2.992021
2,2,1.0,10200.0,0,0,0,-2.085600,57.152140,"Stylish 2 Bedroom First Floor Apartment, Spaci...",59.708713,0.556872,1.053817,1.992315,2.753622e-08,0.902443,0.914426,0.135444,4.847744e-01,2.911739
3,2,1.0,9000.0,0,0,0,-2.098261,57.145840,"No Agent Fees, Students Can Enquire, Property ...",86.670721,2.999418,1.997477,1.998880,9.999946e-01,0.000000,0.000000,0.998626,9.999392e-01,2.999777
4,2,2.0,10200.0,0,0,0,-2.095850,57.153930,"Double glazing, Shower, Washing machine, Telev...",73.573304,2.565222,1.414026,1.838475,5.623989e-01,0.000000,0.000000,0.717756,9.332635e-01,2.999338


# Split data

In [152]:
from sklearn.model_selection import train_test_split

# Assuming 'df' is your pandas DataFrame
train_val, test = train_test_split(full_df, test_size=0.1, random_state=42)  # 10% for test set
train, val = train_test_split(train_val, test_size=0.2, random_state=42)


In [153]:
test.shape, val.shape, train.shape

((6247, 19), (11244, 19), (44972, 19))

In [155]:
test.to_csv("data/test.csv")
train.to_csv("data/train.csv")
val.to_csv("data/val.csv")